In [ ]:
import sys
sys.path.append("../")

In [ ]:
import os

import pandas as pd
import joblib

from sklearn.ensemble import RandomForestRegressor

import os.path as op
import random

import matplotlib.pyplot as plt
import yaml

from agents.matchmanager import MatchManager
from agents.ml.regressionMulti import RegressionMultiAgent
from agents import GreedyAgent,RandomAgent
from agents.matchmanager import MatchManager
from agents.ml.classifier import ClassifierAgent
from agents.ml.regression import RegressionAgent

from core.const import RED, BLUE
from scenarios import scenarioJunction

from core.vectors import vectorAction, vectorActionInfo, vectorBoard, vectorBoardInfo
from core.game import GameBoard, GameState, vectorState, vectorStateInfo


import warnings

%matplotlib inline
plt.rcParams['figure.figsize'] = (12,4)

In [ ]:
with open(op.join('.', '../logger.config.yaml'), 'r') as stream:
    config = yaml.load(stream, Loader=yaml.FullLoader)
# logging.config.dictConfig(config)

In [ ]:
raw = pd.read_pickle('../data.2021-02-22.scenarioJunction.pkl.gz')\
        .drop(['meta_scenario', 'meta_p_red', 'meta_p_blue', 'meta_seed'], axis=1, errors="ignore")

In [ ]:
df = raw.dropna(axis=1, how='all')

In [ ]:
df_red = df[df['action_team'] == 'red'].copy().drop('action_team', axis=1)
df_blue = df[df['action_team'] == 'blue'].copy().drop('action_team', axis=1)

In [ ]:
df_red['label'] = df_red['winner'].apply(lambda x: 1 if x == 'red' else -1)
df_blue['label'] = df_blue['winner'].apply(lambda x: 1 if x == 'blue' else -1)

In [ ]:
df_red_move=df_red.loc[((df_red['action_type_Move'] == True)|(df_red['action_type_LoadInto'] == True))].copy()
df_red_attack=df_red.loc[((df_red['action_type_Attack'] == True)| (df_red['action_type_AttackGround'] == True) |(df_red['action_type_AttackRespond'] == True))].copy()
df_red_pass=df_red.loc[((df_red['action_type_Pass'] == True)| (df_red['action_type_PassFigure'] == True) |(df_red['action_type_PassTeam'] == True) | (df_red['action_type_PassRespond'] == True))].copy()

In [ ]:
df_blue_move=df_blue.loc[((df_blue['action_type_Move'] == True)|(df_blue['action_type_LoadInto'] == True))].copy()
df_blue_attack=df_blue.loc[((df_blue['action_type_Attack'] == True)| (df_blue['action_type_AttackGround'] == True) |(df_blue['action_type_AttackRespond'] == True))].copy()
df_blue_pass=df_blue.loc[((df_blue['action_type_Pass'] == True)| (df_blue['action_type_PassFigure'] == True) |(df_blue['action_type_PassTeam'] == True) | (df_blue['action_type_PassRespond'] == True))].copy()

In [ ]:
X_red_move = df_red_move.drop(['winner', 'label'], axis=1)
y_red_move = df_red_move['label']

X_blue_move = df_blue_move.drop(['winner', 'label'], axis=1)
y_blue_move = df_blue_move['label']

In [ ]:
X_red_attack = df_red_attack.drop(['winner', 'label'], axis=1)
y_red_attack  = df_red_attack['label']

X_blue_attack  = df_blue_attack.drop(['winner', 'label'], axis=1)
y_blue_attack  = df_blue_attack['label']

In [ ]:
X_red_pass = df_red_pass.drop(['winner', 'label'], axis=1)
y_red_pass  = df_red_pass['label']

X_blue_pass  = df_blue_pass.drop(['winner', 'label'], axis=1)
y_blue_pass  = df_blue_pass['label']

In [ ]:
X = pd.concat([X_red_move, X_blue_move,X_red_attack,X_blue_attack,X_red_pass,X_blue_pass])
y = pd.concat([y_red_move, y_blue_move,y_red_attack,y_blue_attack,y_red_pass,y_blue_pass])

In [ ]:
rfr_red_move = RandomForestRegressor()
rfr_blue_move = RandomForestRegressor()
rfr_red_attack = RandomForestRegressor()
rfr_blue_attack = RandomForestRegressor()
rfr_red_pass = RandomForestRegressor()
rfr_blue_pass = RandomForestRegressor()
rfr = RandomForestRegressor()

rfr_red_move.fit(X_red_move, y_red_move)
rfr_red_attack.fit(X_red_attack, y_red_attack)
rfr_red_pass.fit(X_red_pass, y_red_pass)

rfr_blue_move.fit(X_blue_move, y_blue_move)
rfr_blue_attack.fit(X_blue_attack, y_blue_attack)
rfr_blue_pass.fit(X_blue_pass, y_blue_pass)


rfr.fit(X, y)

joblib.dump(rfr_red_move, '../models/Junction_RandomForestRegressor_red_move_20210221.joblib')
joblib.dump(rfr_red_attack, '../models/Junction_RandomForestRegressor_red_attack_20210221.joblib')
joblib.dump(rfr_red_pass, '../models/Junction_RandomForestRegressor_red_pass_20210221.joblib')


joblib.dump(rfr_blue_move, '../models/Junction_RandomForestRegressor_blue_move_20210221.joblib')
joblib.dump(rfr_blue_attack, '../models/Junction_RandomForestRegressor_blue_attack_20210221.joblib')
joblib.dump(rfr_blue_pass, '../models/Junction_RandomForestRegressor_blue_pass_20210221.joblib')

joblib.dump(rfr, '../models/Junction_RandomForestRegressor_20210221.joblib')

In [ ]:
seed=52

In [ ]:
red_models = [
    ('../models/Junction_RandomForestRegressor_red_attack_20210221.joblib'),
    ('../models/Junction_RandomForestRegressor_red_move_20210221.joblib'),
    ('../models/Junction_RandomForestRegressor_red_pass_20210221.joblib'),

]
blue_models = [
    ('../models/Junction_RandomForestRegressor_blue_attack_20210221.joblib'),
    ('../models/Junction_RandomForestRegressor_blue_move_20210221.joblib'),
    ('../models/Junction_RandomForestRegressor_blue_pass_20210221.joblib'),

]

In [ ]:
red_models = [
    ('gre', ''),
    ('cls', '../models/Junction_RandomForestClassifier_red.joblib'),
    ('cls', '../models/Junction_RandomForestClassifier.joblib'),
    ('reg', '../models/Junction_RandomForestRegressor_red.joblib'),
    ('reg', '../models/Junction_RandomForestRegressor.joblib'),
    ('ran',''),
    ('regMulti','../models/Junction_RandomForestRegressor_red_attack_20210221.joblib','../models/Junction_RandomForestRegressor_red_move_20210221.joblib''../models/Junction_RandomForestRegressor_red_pass_20210221.joblib')
]
blue_models = [
    ('gre', ''),
    ('cls', '../models/Junction_RandomForestClassifier_blue.joblib'),
    ('cls', '../models/Junction_RandomForestClassifier.joblib'),
    ('reg', '../models/Junction_RandomForestRegressor_blue.joblib'),
    ('reg', '../models/Junction_RandomForestRegressor.joblib'),
    ('ran',''),
    ('regMulti','../models/Junction_RandomForestRegressor_blue_attack_20210221.joblib','../models/Junction_RandomForestRegressor_blue_move_20210221.joblib''../models/Junction_RandomForestRegressor_blue_pass_20210221.joblib')


]

In [ ]:
def game():
    board, state = scenarioJunction()
    
    print('seed:   ', seed)

    playerRed = RegressionMultiAgent(RED,red_models[0],red_models[1],red_models[2],seed)
    playerBlue = RegressionMultiAgent(BLUE,blue_models[0],blue_models[1],blue_models[2],seed)

    mm = MatchManager(' ', playerRed, playerBlue, board, state, seed=seed)
    while not mm.end:
        mm.nextStep()
    
    print('winner: ', mm.winner)

    return mm

In [ ]:
def player(team:str, data:tuple, seed:int):
    if data[0] == 'gre':
        print(f'{team:5}-> GreedyAgent')
        return GreedyAgent(team, seed=seed)
    if data[0] == 'cls':
        print(f'{team:5}-> ClassifierAgent')
        return ClassifierAgent(team, data[1], seed=seed)
    if data[0] == 'reg':
        print(f'{team:5}-> RegressionAgent')
        return RegressionAgent(team, data[1], seed=seed)
    if data[0]=='ran':
        print(f'{team:5}-> RandomAgent')
        return RandomAgent(team, seed=seed)
        
    
    return None

In [ ]:
def gameJunction(seed:int, red:tuple, blue:tuple) -> MatchManager:
    board, state = scenarioJunction()
    
    print('seed:   ', seed)
    
    playerRed = player(RED, red, seed)
    playerBlue = player(BLUE, blue, seed)

    mm = MatchManager(' ', playerRed, playerBlue, board, state, seed=seed)
    while not mm.end:
        mm.nextStep()
    
    print('winner: ', mm.winner)

    return mm

In [ ]:
mm=game()

In [ ]:
def prepareDataFrames(mm:MatchManager):
    df_states = pd.DataFrame(
        columns=vectorStateInfo(),
        data=[vectorState(x) for x in mm.states_history]
    )

    df_actions = pd.DataFrame(
        columns=vectorActionInfo(),
        data=[vectorAction(x) for x in mm.actions_history]
    )
    df_red = mm.red.createDataFrame()
    df_blue = mm.blue.createDataFrame()
    
    df_red = df_red.set_index('count')
    df_blue = df_blue.set_index('count')

    return df_red, df_blue, df_states, df_actions

In [ ]:
def plotEntropy(df, color:str):
    plt.figure()
    df.score.plot(color=color, marker='.')
    df.entropy.plot(color='purple')

    for i, row in df[['score','action']].reset_index().iterrows():
        dy, ha, va = (-0, 'center', 'top') if 'Respond' in row.action else (10, 'center', 'bottom')
        plt.annotate(row.action, (row['count'], row.score), ha=ha, va=va, rotation=90, textcoords="offset points", xytext=(0,dy))
        
    for x in df[['turn']].reset_index().groupby('turn').max().values.tolist():
        plt.axvline(x=x[0]+0.5, color='orange')

In [ ]:
def plotContrast(df):
    df[df['team']=='red']['entropy'].plot(color='red'), df[df['team']=='blue']['entropy'].plot(color='blue')

In [ ]:
def playAndAnalyze(seed:int, red_data, blue_data):  
    mm = gameJunction(seed, red_data, blue_data)
        
    df_red, df_blue, _, _ = prepareDataFrames(mm)
    
    plotEntropy(df_red, 'red')
    plotEntropy(df_blue, 'blue')
    
    return df_red, df_blue

In [ ]:
df_red, df_blue, _, _ = prepareDataFrames(mm)


In [ ]:
plotEntropy(df_red, 'red')
plotEntropy(df_blue, 'blue')

In [ ]:
s = random.randint(0, 1000000)
r = random.randint(0, len(red_models)-1)
b = random.randint(0, len(blue_models)-1)

playAndAnalyze(s, red_models[r], blue_models[b]);